In [2]:
import folium
import pandas as pd
import json
from folium import plugins
import geocoder
from geopy.distance import great_circle

In [3]:
with open('./전국_정신건강관련기관_위치정보.json', 'r',encoding='utf-8') as f:
    loc_data = json.load(f)
    
loc_arr = loc_data['data']
loc_arr

df = pd.DataFrame(loc_arr)


df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')

df.loc[df['기관구분']=='정신병원', '기관구분'] = '병원'

df.index = range(len(df))
df

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,0,공공,상담소,(주)마음꼭 차심리상담센터,경기도 성남시 분당구 야탑동 351 기억력센터 2층 (야탑로 59 분당차병원),37.410250,127.125250,031-780-5437,None
1,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
2,2,공공,상담소,가능성연구소,부산시 해운대구 세실로79 부산해운대대교사옥 6층,35.171647,129.176440,051-704-8610,None
3,3,공공,상담소,가톨릭대학교 은평성모병원,서울 은평구 진관동 93-6,37.633251,126.916547,02-1811-7755,None
4,4,공공,상담소,강원대학교병원 정신건강의학과,강원도 춘천시 백령로 156 강원대학교병원 정신건강의학과,37.874701,127.744801,033-258-2437,None
...,...,...,...,...,...,...,...,...,...
4327,4706,민간,병원,실버한방병원,서울특별시 금천구 독산로 362 (독산동),37.479089,126.904289,02-857-2001,None
4328,4707,민간,병원,의료법인 영명의료재단 혜민한방병원,"경기도 시흥시 군자로 526 도일그랜드프라자 302,401,501호 (거모동)",37.343354,126.785183,031-504-7272,None
4329,4708,민간,병원,한도한방병원,경기도 의정부시 입석로 36 (가능동),37.755883,127.021652,031-877-6081,None
4330,4709,민간,병원,혜민한방병원,충청남도 태안군 태안읍 정주내1길 13-0 혜민한방병원,36.745075,126.304532,041-400-7171,None


In [15]:
class MappingByCoord:

    
    def __init__(self, df, lat, lng, dist = 5):

        self.df = df
        self.lat = lat
        self.lng = lng
        self.dist = dist
        
    def setRectangler(self):

        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lng_min = self.lng - 0.015 * self.dist
        lng_max = self.lng + 0.015 * self.dist

        self.points = [[lat_min, lng_min], [lat_max, lng_max]]

        result = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lng_min) &
            (self.df['경도'] < lng_max)
        ]
        result.index = range(len(result))

        return result    
        

    def setCircle(self):

        tmp = self.setRectangler()

        center = (self.lat, self.lng)

        result = pd.DataFrame()

        for index, row in tmp.iterrows():
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result
        
    def MappingInRectangler(self, df):


        m = folium.Map(location=[self.lat, self.lng], zoom_start=14)
        

        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            folium.Marker(location=[lat_now, lng_now],
                          radius=15,
                          tooltip=row['기관명']).add_to(m)

        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(m)

        return m 
    
    def MappingInCircle(self, df):

        m = folium.Map(location=[self.lat, self.lng], width='70%', height='100%', zoom_start=13, tiles=None)
        folium.TileLayer('openstreetmap', name='구분').add_to(m)

        mcg = folium.plugins.MarkerCluster(control=False)
        m.add_child(mcg)
        sangdam = folium.plugins.FeatureGroupSubGroup(mcg, "상담소")
        center_ = folium.plugins.FeatureGroupSubGroup(mcg, "센터")
        ins = folium.plugins.FeatureGroupSubGroup(mcg, "시설")
        hos = folium.plugins.FeatureGroupSubGroup(mcg, "병원")
        bogun = folium.plugins.FeatureGroupSubGroup(mcg, "보건소")
        
        m.add_child(sangdam)
        m.add_child(center_)
        m.add_child(ins)
        m.add_child(hos)
        m.add_child(bogun)
        
        for idx, row in df.iterrows():

            lat_now = row['위도']
            lng_now = row['경도']

            
            div = ['상담소', '센터', '시설', '병원', '보건소']
            
            
            if row['기관구분'] == div[0]:
                sangdam.add_child(folium.Marker([lat_now, lng_now], icon=folium.Icon(color='pink'), radius=15, tooltip=row['기관명']))
                
            elif row['기관구분'] == div[1]:
                center_.add_child(folium.Marker([lat_now, lng_now], icon=folium.Icon(color='green'), radius=15, tooltip=row['기관명']))
                
            elif row['기관구분'] == div[2]:
                ins.add_child(folium.Marker([lat_now, lng_now], icon=folium.Icon(color='blue'), radius=15, tooltip=row['기관명']))
                
            elif row['기관구분'] ==div[3]:
                hos.add_child(folium.Marker([lat_now, lng_now], icon=folium.Icon(color='purple'), radius=15, tooltip=row['기관명']))
                
            else:
                bogun.add_child(folium.Marker([lat_now, lng_now], icon=folium.Icon(color='orange'), radius=15, tooltip=row['기관명']))
            

                
        folium.Circle(radius=dist * 1000,
                      location=[lat, lng],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m

In [20]:
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng
    
lat = mylocation[0]
lng = mylocation[1]
dist = 5

In [21]:
lat_min = lat - 0.01 * dist
lat_max = lat + 0.01 * dist

lng_min = lng - 0.015 * dist
lng_max = lng + 0.015 * dist

points = [[lat_min, lng_min], [lat_max, lng_max]]

result = df.loc[
    (df['위도'] > lat_min) &
    (df['위도'] < lat_max) &
    (df['경도'] > lng_min) &
    (df['경도'] < lng_max)
]
result.index = range(len(result))
    
result = tmp


tmp 


,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
1,9,공공,상담소,고려대학교 학생상담센터,서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114,37.588703,127.031698,02-3290-1540,None
2,12,공공,상담소,광화문 마음공간 심리상담센터,서울시 종로구 사직로 8길 34 경희궁의 아침 3단지 오피스텔 211호,37.573588,126.972426,02-720-7127,None
3,14,공공,상담소,김연 심리상담 연구소,(04037) 서울 마포구 양화로 59 (서교동) 화승리버스텔 5층 502호,37.550659,126.914886,02-336-0707,None
4,18,공공,상담소,나사랑심리상담센터,서울시 성북구 삼선동2가 22번지 현명빌딩 302호,37.588535,127.010128,02-747-2131,None
...,...,...,...,...,...,...,...,...,...
326,4583,민간,병원,홍대성모정신건강의학과의원,서울특별시 마포구 양화로 166 미래프라자 12(일부)층 (동교동),37.556733,126.924171,02-336-0166,None
327,4590,민간,병원,홍정신건강의학과의원,"서울특별시 강남구 학동로 411 2층 204호 (청담동, 원일빌딩)",37.518102,127.044728,02-568-0138,None
328,4592,민간,병원,홍제정신건강의학과의원,"서울특별시 서대문구 통일로 476 2층 204호 (홍제동, 원일아파트)",37.590756,126.942435,02-394-0887,None
329,4602,민간,병원,황규혁정신건강의학과의원,서울특별시 강북구 도봉로 34 5층 (미아동),37.612080,127.030735,02-945-7751,None


In [22]:
center = (lat, lng)

result_circle = pd.DataFrame()

for index, row in tmp.iterrows():
    point = (row['위도'], row['경도'])
    d = great_circle(center, point).kilometers
    if d <= dist:
        result_circle = pd.concat([result_circle, tmp.iloc[index, :].to_frame().T])
            
result_circle.index = range(len(result_circle))
    
result_circle

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
1,12,공공,상담소,광화문 마음공간 심리상담센터,서울시 종로구 사직로 8길 34 경희궁의 아침 3단지 오피스텔 211호,37.573588,126.972426,02-720-7127,None
2,18,공공,상담소,나사랑심리상담센터,서울시 성북구 삼선동2가 22번지 현명빌딩 302호,37.588535,127.010128,02-747-2131,None
3,41,공공,상담소,마음돌봄연구소,"서울특별시 중구 동호로 186-7 501호 (신당동, MJ빌딩)",37.554605,127.011505,010-8290-1624,None
4,43,공공,상담소,마음뜨락 임상심리센터,서울 성동구 한림말길 48 옥수중앙하이츠APT상가 3층,37.54101,127.016197,02-2291-7233,None
...,...,...,...,...,...,...,...,...,...
163,4500,민간,병원,해람정신건강의학과의원,서울특별시 마포구 양화로 178 4층 (동교동),37.557477,126.925233,02-723-8189,None
164,4507,민간,병원,해민정신건강의학과의원,서울특별시 성동구 왕십리로 309 3층 303호 (행당동),37.56135,127.034734,02-2292-1535,None
165,4510,민간,병원,해솔정신건강의학과의원,서울특별시 마포구 백범로 13 신촌르메이에르타운II 3층 303호 (노고산동),37.553389,126.937475,02-6357-7575,None
166,4583,민간,병원,홍대성모정신건강의학과의원,서울특별시 마포구 양화로 166 미래프라자 12(일부)층 (동교동),37.556733,126.924171,02-336-0166,None


In [5]:
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng
    
lat = mylocation[0]
lng = mylocation[1]
dist = 5

In [6]:
df

list_filter = setRectangler(df, lat, lng)


list_filter

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,1,공공,상담소,(주)마음연구소 부설 마음iN심리상담센터,서울 종로구 종로3길 34 6층,37.571875,126.979481,02-722-2883,None
1,9,공공,상담소,고려대학교 학생상담센터,서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114,37.588703,127.031698,02-3290-1540,None
2,12,공공,상담소,광화문 마음공간 심리상담센터,서울시 종로구 사직로 8길 34 경희궁의 아침 3단지 오피스텔 211호,37.573588,126.972426,02-720-7127,None
3,18,공공,상담소,나사랑심리상담센터,서울시 성북구 삼선동2가 22번지 현명빌딩 302호,37.588535,127.010128,02-747-2131,None
4,28,공공,상담소,동덕여자대학교 일반대학원,서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과,37.606815,127.041176,02-940-4498,None
...,...,...,...,...,...,...,...,...,...
245,4535,민간,병원,행복찾기정신과의원,서울특별시 강북구 도봉로 52 연이빌딩 2(M)층 (미아동),37.613131,127.029462,02-981-7772,None
246,4536,민간,병원,행복한가정의학과의원,서울특별시 동대문구 한천로 242 2층 (전농동),37.582500,127.068606,02-2245-3100,None
247,4592,민간,병원,홍제정신건강의학과의원,"서울특별시 서대문구 통일로 476 2층 204호 (홍제동, 원일아파트)",37.590756,126.942435,02-394-0887,None
248,4602,민간,병원,황규혁정신건강의학과의원,서울특별시 강북구 도봉로 34 5층 (미아동),37.612080,127.030735,02-945-7751,None


In [7]:
def setCircle(df, lat, lng, dist=5):

    tmp = list_filter

    center = (lat, lng)

    result = pd.DataFrame()

    for index, row in tmp.iterrows():
        point = (row['위도'], row['경도'])
        d = great_circle(center, point).kilometers
        if d <= dist:
            result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
            
        result.index = range(len(result))
            
        return result

In [10]:
circle_filter = setCircle(df, lat, lng, dist=5)

circle_filter.index = range(len(circle_filter))

print(circle_filter)

    index 공공/민간 기관구분            기관명  \
0       9    공공  상담소   고려대학교 학생상담센터   
1      18    공공  상담소      나사랑심리상담센터   
2      28    공공  상담소  동덕여자대학교 일반대학원   
3      75    공공  상담소       서울해바라기센터   
4     149    공공  상담소            푸른존   
..    ...   ...  ...            ...   
114  4416    민간   병원    하나정신건강의학과의원   
115  4502    민간   병원     해맑은이비인후과의원   
116  4535    민간   병원      행복찾기정신과의원   
117  4602    민간   병원   황규혁정신건강의학과의원   
118  4617    민간   병원   휴앤힐정신건강의학과의원   

                                                    주소         위도          경도  \
0                    서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114  37.588703  127.031698   
1                         서울시 성북구 삼선동2가 22번지 현명빌딩 302호  37.588535  127.010128   
2              서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과  37.606815  127.041176   
3    서울특별시 종로구 대학로 101 서울대학교병원 함춘회관 지하 101호 / 서울특별시...  37.580452  126.997196   
4                                서울시 강북구 솔매로49길 6(미아동)  37.626831  127.027013   
..                                           

In [13]:
a = circle_filter.iloc[0]
a

index                                    9
공공/민간                                   공공
기관구분                                   상담소
기관명                           고려대학교 학생상담센터
주소       서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114
위도                               37.588703
경도                              127.031698
전화번호                          02-3290-1540
홈페이지                                  None
Name: 0, dtype: object

In [14]:
a['주소']

'서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114'

In [9]:
dict_circle = circle_filter.to_dict()
dict_circle

{'index': {0: 9,
  1: 18,
  2: 28,
  3: 75,
  4: 149,
  5: 153,
  6: 189,
  7: 202,
  8: 232,
  9: 256,
  10: 265,
  11: 284,
  12: 300,
  13: 383,
  14: 384,
  15: 423,
  16: 495,
  17: 541,
  18: 582,
  19: 583,
  20: 642,
  21: 690,
  22: 833,
  23: 839,
  24: 863,
  25: 865,
  26: 867,
  27: 881,
  28: 937,
  29: 1022,
  30: 1152,
  31: 1153,
  32: 1154,
  33: 1270,
  34: 1298,
  35: 1333,
  36: 1376,
  37: 1386,
  38: 1408,
  39: 1444,
  40: 1477,
  41: 1479,
  42: 1540,
  43: 1618,
  44: 1625,
  45: 1634,
  46: 1645,
  47: 1804,
  48: 1835,
  49: 1922,
  50: 2134,
  51: 2146,
  52: 2151,
  53: 2239,
  54: 2309,
  55: 2415,
  56: 2437,
  57: 2511,
  58: 2533,
  59: 2541,
  60: 2556,
  61: 2644,
  62: 2669,
  63: 2748,
  64: 2773,
  65: 2786,
  66: 2820,
  67: 2832,
  68: 2861,
  69: 2889,
  70: 2936,
  71: 2999,
  72: 3001,
  73: 3065,
  74: 3134,
  75: 3171,
  76: 3173,
  77: 3203,
  78: 3212,
  79: 3215,
  80: 3230,
  81: 3232,
  82: 3244,
  83: 3254,
  84: 3290,
  85: 3301,
  8

In [4]:
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng
    
lat = mylocation[0]
lng = mylocation[1]
dist = 5

mbc = MappingByCoord(df, lat, lng, dist)

result_radius = mbc.setCircle()

mymap = mbc.MappingInCircle(result_radius)
folium.Marker(location=mylocation, popup='현재 나의 위치', icon=folium.Icon(color='red', icon='star')).add_to(mymap)
plugins.LocateControl().add_to(mymap)
plugins.Geocoder(position='bottomright', collapsed=True, add_marker=True).add_to(mymap)

folium.LayerControl(collapsed=True, position='bottomright').add_to(mymap)
mymap.layer_name = '구분'
mymap


In [114]:
mymap.save('./나의 5km 반경내 병원 좌표.html')

In [123]:
m1 = folium.Map(location=mylocation, tiles="Stamen Terrain", zoom_start=13)

folium.Marker(mylocation, popup="Camp Muir").add_to(m1)

m1.add_child(folium.ClickForMarker(popup="Waypoint"))


m1


In [122]:
m1 = folium.Map(location=mylocation, tiles="Stamen Terrain", zoom_start=13)

m1.add_child(folium.LatLngPopup())

m1


In [6]:
import folium
import pandas as pd
import json
from folium import plugins
import geocoder
from geopy.distance import great_circle

In [7]:
from jinja2 import Template
import folium


class ClickForOneMarker(folium.ClickForMarker):
    _template = Template(u"""
            {% macro script(this, kwargs) %}
                var new_mark = L.marker();
                function newMarker(e){
                    new_mark.setLatLng(e.latlng).addTo({{this._parent.get_name()}});
                    new_mark.dragging.enable();
                    new_mark.on('dblclick', function(e){ {{this._parent.get_name()}}.removeLayer(e.target)})
                    var lat = e.latlng.lat.toFixed(4),
                       lng = e.latlng.lng.toFixed(4);
                    new_mark.bindPopup({{ this.popup }});
                    parent.document.getElementById("latitude").value = lat;
                    parent.document.getElementById("longitude").value =lng;
                    };
                {{this._parent.get_name()}}.on('click', newMarker);
            {% endmacro %}
            """)  # noqa

    def __init__(self, popup=None):
        super(ClickForOneMarker, self).__init__(popup)


class LatLngtoForm(folium.LatLngPopup):
    _template = Template(u"""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.popup();
                function latLngPop(e) {
                    data = [e.latlng.lat.toFixed(4) , e.latlng.lng.toFixed(4)];
                    parent.document.getElementById("latitude").value =data[0];
                    parent.document.getElementById("longitude").value =data[1];
                    {{this.get_name()}}
                        .setLatLng(e.latlng)
                        .setContent( '<span data='+data.toString()+' id="locate">여기</span>')
                        .openOn({{this._parent.get_name()}});
                    }
                {{this._parent.get_name()}}.on('click', latLngPop);
            {% endmacro %}
            """)  # noqa

    def __init__(self):
        super(LatLngtoForm, self).__init__()
        self._name = 'LatLngPopup'


In [8]:
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng


m = folium.Map(location=mylocation, tiles="Stamen Terrain", zoom_start=13)


m.add_child(folium.ClickForMarker(popup="Waypoint"))

In [9]:
cfom = ClickForOneMarker()

m.add_child(cfom)

m

In [4]:
user_loc = geocoder.ip('me')
mylocation = user_loc.latlng
    
lat = mylocation[0]
lng = mylocation[1]
dist = 5

lat_min = lat - 0.01 * dist
lat_max = lat + 0.01 * dist

lng_min = lng - 0.015 * dist
lng_max = lng + 0.015 * dist

points = [[lat_min, lng_min], [lat_max, lng_max]]

result = df.loc[
    (df['위도'] > lat_min) &
    (df['위도'] < lat_max) &
    (df['경도'] > lng_min) &
    (df['경도'] < lng_max)
    ]
result.index = range(len(result))

tmp = result

center = (lat, lng)

result_circle = pd.DataFrame()

for index, row in tmp.iterrows():
    point = (row['위도'], row['경도'])
    d = great_circle(center, point).kilometers
    if d <= dist:
        result_circle = pd.concat([result_circle, tmp.iloc[index, :].to_frame().T])

result_circle.index = range(len(result_circle))

result_circle

,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,9,공공,상담소,고려대학교 학생상담센터,서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114,37.588703,127.031698,02-3290-1540,None
1,18,공공,상담소,나사랑심리상담센터,서울시 성북구 삼선동2가 22번지 현명빌딩 302호,37.588535,127.010128,02-747-2131,None
2,28,공공,상담소,동덕여자대학교 일반대학원,서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과,37.606815,127.041176,02-940-4498,None
3,75,공공,상담소,서울해바라기센터,서울특별시 종로구 대학로 101 서울대학교병원 함춘회관 지하 101호 / 서울특별시...,37.580452,126.997196,070-4287-0369,None
4,149,공공,상담소,푸른존,서울시 강북구 솔매로49길 6(미아동),37.626831,127.027013,02-944-5811,None
...,...,...,...,...,...,...,...,...,...
114,4416,민간,병원,하나정신건강의학과의원,서울특별시 성북구 동소문로 97 5층 2호 (동소문동6가),37.592798,127.016025,922-3075,None
115,4502,민간,병원,해맑은이비인후과의원,서울특별시 성북구 종암로 115 지상4층 (종암동),37.600383,127.033573,02-922-1282,None
116,4535,민간,병원,행복찾기정신과의원,서울특별시 강북구 도봉로 52 연이빌딩 2(M)층 (미아동),37.613131,127.029462,02-981-7772,None
117,4602,민간,병원,황규혁정신건강의학과의원,서울특별시 강북구 도봉로 34 5층 (미아동),37.61208,127.030735,02-945-7751,None


In [8]:
array_result = result_circle.values

array_result

array([[9, '공공', '상담소', ..., 127.0316983312, '02-3290-1540', None],
       [18, '공공', '상담소', ..., 127.0101277234, '02-747-2131', None],
       [28, '공공', '상담소', ..., 127.0411763018, '02-940-4498', None],
       ...,
       [4535, '민간', '병원', ..., 127.029462, '02-981-7772', None],
       [4602, '민간', '병원', ..., 127.0307347, '02-945-7751', None],
       [4617, '민간', '병원', ..., 127.0295141, '922-1183',
        'https://hueandheal.modoo.at/']], dtype=object)

In [9]:
result_list = array_result.tolist()

result_list

[[9,
  '공공',
  '상담소',
  '고려대학교 학생상담센터',
  '서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114',
  37.5887034224,
  127.0316983312,
  '02-3290-1540',
  None],
 [18,
  '공공',
  '상담소',
  '나사랑심리상담센터',
  '서울시 성북구 삼선동2가 22번지 현명빌딩 302호',
  37.5885353347,
  127.0101277234,
  '02-747-2131',
  None],
 [28,
  '공공',
  '상담소',
  '동덕여자대학교 일반대학원',
  '서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과',
  37.6068148328,
  127.0411763018,
  '02-940-4498',
  None],
 [75,
  '공공',
  '상담소',
  '서울해바라기센터',
  '서울특별시 종로구 대학로 101 서울대학교병원 함춘회관 지하 101호 / 서울특별시 종로구 대학로8가길 56 동숭빌딩 2층',
  37.5804523853,
  126.9971964379,
  '070-4287-0369',
  None],
 [149,
  '공공',
  '상담소',
  '푸른존',
  '서울시 강북구 솔매로49길 6(미아동)',
  37.6268313147,
  127.0270125407,
  '02-944-5811',
  None],
 [153,
  '공공',
  '상담소',
  '한국상담임상심리센터',
  '서울 종로구 삼일대로 30길 23 , 517호(비즈웰종로오피스텔)',
  37.5744754999,
  126.9887757806,
  '010-2765-2512',
  None],
 [189,
  '공공',
  '상담소',
  '아하가족성장연구소',
  '서울 종로구 창의문로10길 11  4층(부암동, 일민기념관)',
  37.5938781053,
  126.9642539379,
  '02-379-8558',
  'http:

In [11]:
result_dict = result_circle.to_dict('list')

result_dict

{'index': [9,
  18,
  28,
  75,
  149,
  153,
  189,
  202,
  232,
  256,
  265,
  284,
  300,
  383,
  384,
  423,
  495,
  541,
  582,
  583,
  642,
  690,
  833,
  839,
  863,
  865,
  867,
  881,
  937,
  1022,
  1152,
  1153,
  1154,
  1270,
  1298,
  1333,
  1376,
  1386,
  1408,
  1444,
  1477,
  1479,
  1540,
  1618,
  1625,
  1634,
  1645,
  1804,
  1835,
  1922,
  2134,
  2146,
  2151,
  2239,
  2309,
  2415,
  2437,
  2511,
  2533,
  2541,
  2556,
  2644,
  2669,
  2748,
  2773,
  2786,
  2820,
  2832,
  2861,
  2889,
  2936,
  2999,
  3001,
  3065,
  3134,
  3171,
  3173,
  3203,
  3212,
  3215,
  3230,
  3232,
  3244,
  3254,
  3290,
  3301,
  3357,
  3412,
  3473,
  3490,
  3616,
  3645,
  3672,
  3686,
  3739,
  3758,
  3804,
  3813,
  3816,
  3835,
  3933,
  3954,
  4037,
  4059,
  4084,
  4092,
  4095,
  4107,
  4114,
  4196,
  4243,
  4247,
  4279,
  4343,
  4416,
  4502,
  4535,
  4602,
  4617],
 '공공/민간': ['공공',
  '공공',
  '공공',
  '공공',
  '공공',
  '공공',
  '공공',
  '공공',

In [13]:
a = result_circle.to_json(orient='records', force_ascii=False)

a

'[{"index":9,"공공\\/민간":"공공","기관구분":"상담소","기관명":"고려대학교 학생상담센터","주소":"서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114","위도":37.5887034224,"경도":127.0316983312,"전화번호":"02-3290-1540","홈페이지":null},{"index":18,"공공\\/민간":"공공","기관구분":"상담소","기관명":"나사랑심리상담센터","주소":"서울시 성북구 삼선동2가 22번지 현명빌딩 302호","위도":37.5885353347,"경도":127.0101277234,"전화번호":"02-747-2131","홈페이지":null},{"index":28,"공공\\/민간":"공공","기관구분":"상담소","기관명":"동덕여자대학교 일반대학원","주소":"서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과","위도":37.6068148328,"경도":127.0411763018,"전화번호":"02-940-4498","홈페이지":null},{"index":75,"공공\\/민간":"공공","기관구분":"상담소","기관명":"서울해바라기센터","주소":"서울특별시 종로구 대학로 101 서울대학교병원 함춘회관 지하 101호 \\/ 서울특별시 종로구 대학로8가길 56 동숭빌딩 2층","위도":37.5804523853,"경도":126.9971964379,"전화번호":"070-4287-0369","홈페이지":null},{"index":149,"공공\\/민간":"공공","기관구분":"상담소","기관명":"푸른존","주소":"서울시 강북구 솔매로49길 6(미아동)","위도":37.6268313147,"경도":127.0270125407,"전화번호":"02-944-5811","홈페이지":null},{"index":153,"공공\\/민간":"공공","기관구분":"상담소","기관명":"한국상담임상심리센터","주소":"서울 종로구 삼일대로 30길 23 , 517호(비즈웰종로오피스텔)","위도":

In [14]:
b = result_circle.query()

b

TypeError: query() missing 1 required positional argument: 'expr'

In [15]:
result_circle


,index,공공/민간,기관구분,기관명,주소,위도,경도,전화번호,홈페이지
0,9,공공,상담소,고려대학교 학생상담센터,서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114,37.588703,127.031698,02-3290-1540,None
1,18,공공,상담소,나사랑심리상담센터,서울시 성북구 삼선동2가 22번지 현명빌딩 302호,37.588535,127.010128,02-747-2131,None
2,28,공공,상담소,동덕여자대학교 일반대학원,서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과,37.606815,127.041176,02-940-4498,None
3,75,공공,상담소,서울해바라기센터,서울특별시 종로구 대학로 101 서울대학교병원 함춘회관 지하 101호 / 서울특별시...,37.580452,126.997196,070-4287-0369,None
4,149,공공,상담소,푸른존,서울시 강북구 솔매로49길 6(미아동),37.626831,127.027013,02-944-5811,None
...,...,...,...,...,...,...,...,...,...
114,4416,민간,병원,하나정신건강의학과의원,서울특별시 성북구 동소문로 97 5층 2호 (동소문동6가),37.592798,127.016025,922-3075,None
115,4502,민간,병원,해맑은이비인후과의원,서울특별시 성북구 종암로 115 지상4층 (종암동),37.600383,127.033573,02-922-1282,None
116,4535,민간,병원,행복찾기정신과의원,서울특별시 강북구 도봉로 52 연이빌딩 2(M)층 (미아동),37.613131,127.029462,02-981-7772,None
117,4602,민간,병원,황규혁정신건강의학과의원,서울특별시 강북구 도봉로 34 5층 (미아동),37.61208,127.030735,02-945-7751,None


In [17]:
result_json = result_circle.to_json(orient= 'records', force_ascii=False)
list = json.loads(result_json)
list

[{'index': 9,
  '공공/민간': '공공',
  '기관구분': '상담소',
  '기관명': '고려대학교 학생상담센터',
  '주소': '서울특별시 성북구 안암로 145 고려대학교 중앙광장 B114',
  '위도': 37.5887034224,
  '경도': 127.0316983312,
  '전화번호': '02-3290-1540',
  '홈페이지': None},
 {'index': 18,
  '공공/민간': '공공',
  '기관구분': '상담소',
  '기관명': '나사랑심리상담센터',
  '주소': '서울시 성북구 삼선동2가 22번지 현명빌딩 302호',
  '위도': 37.5885353347,
  '경도': 127.0101277234,
  '전화번호': '02-747-2131',
  '홈페이지': None},
 {'index': 28,
  '공공/민간': '공공',
  '기관구분': '상담소',
  '기관명': '동덕여자대학교 일반대학원',
  '주소': '서울시 성북구 화랑로13길 60 동덕여자대학교 일반대학원 아동·심리학과',
  '위도': 37.6068148328,
  '경도': 127.0411763018,
  '전화번호': '02-940-4498',
  '홈페이지': None},
 {'index': 75,
  '공공/민간': '공공',
  '기관구분': '상담소',
  '기관명': '서울해바라기센터',
  '주소': '서울특별시 종로구 대학로 101 서울대학교병원 함춘회관 지하 101호 / 서울특별시 종로구 대학로8가길 56 동숭빌딩 2층',
  '위도': 37.5804523853,
  '경도': 126.9971964379,
  '전화번호': '070-4287-0369',
  '홈페이지': None},
 {'index': 149,
  '공공/민간': '공공',
  '기관구분': '상담소',
  '기관명': '푸른존',
  '주소': '서울시 강북구 솔매로49길 6(미아동)',
  '위도': 37.6268313147,
  '경도': 127.02